In [0]:
#!python 挺无聊的一个代码lb_8907.163500.py

In [2]:
pwd

'/jupyter_files/magic_mirror'

In [3]:
ls


data/  data.rar  magic_competiton_eda.ipynb  magic_mirror_eda.ipynb  temp.ipynb


In [0]:
#pip install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so"

In [1]:

import pandas as pd
import numpy as np
#显示所有列
pd.set_option("display.max_columns",500)


In [37]:

##文中数据的缺失值全部是\N
def get_null_value(train):
    data=pd.DataFrame()

    dd=[]
    cc=[]
    rr=[]

    for i in train.columns:
        train=train[i].map(lambda x: np.nan if x=='\\n' else x)
        d=len(train)-train[i].count()
        r=(d/len(train))*100
        rate='%.2f%%' % r
#         print('字段名为：',str(i).ljust(10),'缺失值数量:',str(d).ljust(4),'缺失数量占比：',rate) #这里print主要是为了在脚本中观察是否获取到想要的数据，方便调试。
        dd.append(i)
        cc.append(d)
        rr.append(rate)

    data[u'字段名为']=dd
    data[u'缺失值数量']=cc
    data[u'缺失数量占比']=rr
    data=data.sort_values(by='缺失数量占比',ascending=False)
    print(data)
#     return data



In [3]:
path = "data/"
# print(os.listdir(path))
train = pd.read_csv(path+"train.csv",encoding='utf8'
                   ) #100W

In [4]:
train_null=get_null_value(train)#缺失值都是\N替代了

字段名为： user_id    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： listing_id 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： auditing_date 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： due_date   缺失值数量: 0    缺失数量占比： 0.00%
字段名为： due_amt    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： repay_date 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： repay_amt  缺失值数量: 0    缺失数量占比： 0.00%


In [5]:

test = pd.read_csv(path+"test.csv",encoding='utf8',parse_dates=['auditing_date', 'due_date']) #13W


In [19]:
test_null=get_null_value(test)

字段名为： user_id    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： listing_id 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： auditing_date 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： due_date   缺失值数量: 0    缺失数量占比： 0.00%
字段名为： due_amt    缺失值数量: 0    缺失数量占比： 0.00%


In [6]:
train=train.append(test)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [21]:
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1130000 entries, 0 to 129999
Data columns (total 7 columns):
auditing_date    1130000 non-null object
due_amt          1130000 non-null float64
due_date         1130000 non-null object
listing_id       1130000 non-null int64
repay_amt        1000000 non-null object
repay_date       1000000 non-null object
user_id          1130000 non-null int64
dtypes: float64(1), int64(2), object(4)
memory usage: 322.2 MB


In [12]:
train_null=get_null_value(train)#test新增两个字段都是缺失值

            字段名为   缺失值数量  缺失数量占比
4      repay_amt  130000  11.50%
5     repay_date  130000  11.50%
0  auditing_date       0   0.00%
1        due_amt       0   0.00%
2       due_date       0   0.00%
3     listing_id       0   0.00%
6        user_id       0   0.00%


In [10]:
train_null

In [13]:
train.sample(10)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id
170322,2018-04-07,899.7422,2018-05-07,2934452,899.7422,2018-05-07,254213
454981,2018-04-25,216.8220,2018-05-25,3164346,216.822,2018-05-25,288739
428681,2018-04-07,133.9311,2018-05-07,2935929,133.9311,2018-05-07,272446
51505,2018-09-19,148.5317,2018-10-19,4476961,148.5317,2018-10-19,303696
874818,2018-06-04,386.1742,2018-07-04,3626523,\N,\N,660994
591262,2018-09-02,323.8476,2018-10-02,4342226,323.8476,2018-10-02,785881
210467,2018-02-14,168.8729,2018-03-14,2325119,\N,\N,652724
232777,2018-04-25,145.1170,2018-05-25,3161171,145.117,2018-05-25,762545
715575,2018-05-06,193.4560,2018-06-06,3296218,193.456,2018-06-02,489297
93263,2019-02-19 00:00:00,147.9320,2019-03-19 00:00:00,5351637,NaN,NaN,873405


In [14]:
def replace_na_date(x):
  if x=='\\N'or pd.isnull(x):
    return '2200-01-01'
  else:
    return x
def replace_na_amt(x):
  if x=='\\N'or pd.isnull(x):
    return 0
  else :
    return x
train['repay_date']=train['repay_date'].map(replace_na_date)
train['repay_amt']=train['repay_amt'].map(replace_na_amt)

In [15]:
get_null_value(train)#已经没有缺失值

            字段名为  缺失值数量 缺失数量占比
0  auditing_date      0  0.00%
1        due_amt      0  0.00%
2       due_date      0  0.00%
3     listing_id      0  0.00%
4      repay_amt      0  0.00%
5     repay_date      0  0.00%
6        user_id      0  0.00%


In [16]:
train.sample(10)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id
60771,2019-03-19 00:00:00,84.6041,2019-04-19 00:00:00,5454545,0,2200-01-01,880791
359908,2018-07-03,759.7312,2018-08-03,3899342,759.7312,2018-08-03,154385
59764,2019-02-05 00:00:00,159.1147,2019-03-05 00:00:00,5302050,0,2200-01-01,869131
341359,2018-02-03,178.6933,2018-03-03,2207605,178.6933,2018-03-03,659575
604035,2018-03-07,159.1147,2018-04-07,2552030,159.1147,2018-03-20,557867
611984,2018-06-23,395.2154,2018-07-23,3817032,395.2154,2018-07-23,712042
120236,2019-02-10 00:00:00,351.9762,2019-03-10 00:00:00,5316378,0,2200-01-01,839552
751031,2018-01-20,234.6656,2018-02-20,2054101,234.6656,2018-02-19,721600
715444,2018-04-11,1956.5381,2018-05-11,2985587,0,2200-01-01,16685
120497,2018-04-20,266.3328,2018-05-20,3106550,266.3328,2018-05-15,310662


In [17]:
#convert to uint32
for each in ['user_id',	'listing_id']:
  train[each]=train[each].astype(np.uint32)

In [18]:
#convert to datatime
train['auditing_date']=pd.to_datetime(train['auditing_date'],format='%Y-%m-%d')
train['due_date']=pd.to_datetime(train['due_date'],format='%Y-%m-%d')
train['repay_date']=pd.to_datetime(train['repay_date'],format='%Y-%m-%d')

In [19]:
#convert to float32
for each in['due_amt','repay_amt']:
  train[each]=train[each].astype(np.float32)
  

In [34]:
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1130000 entries, 0 to 129999
Data columns (total 7 columns):
auditing_date    1130000 non-null datetime64[ns]
due_amt          1130000 non-null float32
due_date         1130000 non-null datetime64[ns]
listing_id       1130000 non-null uint32
repay_amt        1130000 non-null float32
repay_date       1130000 non-null datetime64[ns]
user_id          1130000 non-null uint32
dtypes: datetime64[ns](3), float32(2), uint32(2)
memory usage: 51.7 MB


In [20]:
train.sample(10)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id
558971,2018-04-01,703.996826,2018-05-01,2849191,703.996826,2018-05-01,448212
107458,2019-03-24,713.339111,2019-04-24,5468256,0.000000,2200-01-01,785731
112423,2018-08-01,160.482407,2018-09-01,4119212,160.482407,2018-09-01,583146
74354,2019-02-14,84.708504,2019-03-14,5333041,0.000000,2200-01-01,449678
678857,2018-05-19,106.458000,2018-06-19,3452896,106.458000,2018-06-15,379713
452371,2018-03-10,133.931107,2018-04-10,2586651,133.931107,2018-04-10,553170
105708,2018-02-24,559.763306,2018-03-24,2408431,559.763306,2018-03-24,38323
30542,2018-04-21,1025.660400,2018-05-21,3114666,0.000000,2200-01-01,140131
7419,2019-03-19,922.245605,2019-04-19,5452953,0.000000,2200-01-01,136359
142639,2018-07-30,249.260101,2018-08-30,4097500,249.260101,2018-08-30,43649


In [21]:
submission = pd.read_csv(path+"/submission/submission.csv",encoding='utf8',parse_dates=[ 'repay_date'],##字符串似乎占空间是最大的
                        dtype={'user_id':np.uint32, 'listing_id':np.uint32,'repay_amt':np.float32
                               }) #398W

In [36]:
submission.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 3 columns):
listing_id    130000 non-null uint32
repay_amt     117675 non-null float32
repay_date    117675 non-null datetime64[ns]
dtypes: datetime64[ns](1), float32(1), uint32(1)
memory usage: 2.0 MB


In [22]:
listing_info = pd.read_csv(path+"listing_info.csv",encoding='utf8',parse_dates=['auditing_date'],
                          dtype={'user_id':np.uint32, 'listing_id':np.uint32,'term':np.uint32,'rate':np.float32,'principal':np.float32 
                                #'due_amt':np.float32,'repay_amt':np.float32
                               })

In [23]:
get_null_value(listing_info)

            字段名为  缺失值数量 缺失数量占比
0        user_id      0  0.00%
1     listing_id      0  0.00%
2  auditing_date      0  0.00%
3           term      0  0.00%
4           rate      0  0.00%
5      principal      0  0.00%


In [0]:
listing_info.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5484891 entries, 0 to 5484890
Data columns (total 6 columns):
user_id          uint32
listing_id       uint32
auditing_date    datetime64[ns]
term             uint32
rate             float32
principal        float32
dtypes: datetime64[ns](1), float32(2), uint32(3)
memory usage: 146.5 MB


In [0]:
listing_info.sample(10)

,user_id,listing_id,auditing_date,term,rate,principal
3948863,594163,4781382,2018-10-29,3,7.2,1610.0
4383776,377263,4565882,2018-10-01,6,8.3,3350.0
4899155,467246,3834112,2018-06-25,3,7.2,630.0
2885252,294758,5232854,2019-01-12,6,6.9,1270.0
2400727,919952,4948312,2018-11-20,6,7.2,3900.0
3394357,464802,767027,2017-08-14,12,9.3,1460.0
765308,137003,3856186,2018-06-27,6,8.0,5540.0
4767476,617770,767202,2017-08-14,12,9.3,1170.0
3159948,658988,2584450,2018-03-10,6,8.0,850.0
3320143,832379,4120966,2018-08-01,9,7.2,630.0


In [24]:
user_info = pd.read_csv(path+"user_info.csv",encoding='utf8',parse_dates=['insertdate','reg_mon'],
                        dtype={'user_id':np.uint32, 'age':np.uint32,
                               })

In [25]:
get_null_value(user_info)

            字段名为  缺失值数量 缺失数量占比
0        user_id      0  0.00%
1        reg_mon      0  0.00%
2         gender      0  0.00%
3            age      0  0.00%
4  cell_province      0  0.00%
5    id_province      0  0.00%
6        id_city      0  0.00%
7     insertdate      0  0.00%


In [0]:
user_info.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954209 entries, 0 to 954208
Data columns (total 8 columns):
user_id          954209 non-null uint32
reg_mon          954209 non-null datetime64[ns]
gender           954209 non-null object
age              954209 non-null uint32
cell_province    954209 non-null object
id_province      954209 non-null object
id_city          954209 non-null object
insertdate       954209 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(4), uint32(2)
memory usage: 264.8 MB


In [31]:
user_info[user_info['cell_province']=='\\N']

,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate
12,583450,2017-07-01,1,20,\N,c08,c08055,2018-08-11
115,768497,2018-01-01,1,24,\N,c08,c08087,2018-02-01
340,727375,2017-11-01,1,19,\N,c17,c17096,2018-04-26
350,629696,2017-08-01,1,25,\N,c26,c26047,2018-12-12
386,916707,2018-11-01,1,36,\N,c05,c05296,2019-03-03
419,830722,2018-04-01,1,22,\N,c17,c17172,2018-12-01
432,770424,2018-01-01,1,33,\N,c02,c02321,2019-02-27
471,770990,2018-01-01,1,22,\N,c08,c08025,2018-05-17
563,551627,2017-06-01,1,28,\N,c17,c17041,2018-05-31
572,777964,2018-01-01,0,22,\N,c06,c06195,2018-04-02


In [26]:
#labelencoder+onehotencoder
from sklearn.preprocessing import LabelEncoder

In [27]:
user_info['gender']=LabelEncoder().fit_transform(user_info['gender'])

In [28]:
user_info['gender']=user_info['gender'].astype(np.uint32)

In [20]:
#fill null value and convert dtype
for each in ['cell_province',	'id_province',	'id_city']:
  user_info[each]=user_info[each].map(lambda x:np.nan if x=='\\N' else x )
  user_info[each].fillna(user_info[each].mode()[0], inplace=True)
  user_info[each]=user_info[each].map(lambda x:x[1:] ).astype(np.uint32)


In [21]:
user_info.head(10)#1代表男，0代表女

,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate
0,483833,2017-04-01,1,19,29,26,26241,2018-12-11
1,156772,2016-05-01,1,31,11,11,11159,2018-02-13
2,173388,2016-05-01,1,34,2,2,2182,2018-08-21
3,199107,2016-07-01,0,25,9,9,9046,2018-06-05
4,122560,2016-03-01,1,23,5,5,5193,2018-04-02
5,550399,2017-06-01,1,30,8,7,7282,2018-04-19
6,414650,2017-02-01,1,53,12,12,23141,2018-10-16
7,275042,2016-10-01,0,23,21,21,21174,2018-07-10
8,184180,2016-06-01,1,29,4,4,4348,2018-02-08
9,817503,2018-03-01,1,25,6,25,25199,2018-11-27


In [0]:
user_info.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954209 entries, 0 to 954208
Data columns (total 8 columns):
user_id          954209 non-null uint32
reg_mon          954209 non-null datetime64[ns]
gender           954209 non-null uint32
age              954209 non-null uint32
cell_province    954209 non-null uint32
id_province      954209 non-null uint32
id_city          954209 non-null uint32
insertdate       954209 non-null datetime64[ns]
dtypes: datetime64[ns](2), uint32(6)
memory usage: 36.4 MB


In [33]:
user_taglist = pd.read_csv(path+"user_taglist.csv",encoding='utf8',parse_dates=['insertdate'],dtype={'user_id':np.uint32})

In [35]:
user_taglist[user_taglist['taglist']=='\\N']

,user_id,taglist,insertdate
16058,153553,\N,2018-03-09
39885,156157,\N,2018-10-28
42516,342080,\N,2018-01-10
44718,24320,\N,2018-01-14
65602,90328,\N,2018-06-07
71003,149938,\N,2018-07-04
84762,167309,\N,2018-03-01
111789,138548,\N,2018-09-12
150806,90080,\N,2019-02-27
220268,153494,\N,2018-03-15


In [38]:
get_null_value(user_taglist)

KeyError: 'user_id'

In [23]:
def property_feature(org):
    tmp=org['taglist'].apply(lambda x:x.split('|')).values
    property_dict={}
    property_list=[]
    for i in tmp:
        property_list+=i
    for i in property_list:
        if i in property_dict:
            property_dict[i]+=1
        else:
            property_dict[i] = 1
#     print(property_dict)
#     print('dict finish')
    def top(x):
        propertys=x.split('|')
        cnt=[property_dict[i] for i in propertys]
        res=sorted(zip(propertys,cnt),key=lambda x:x[1],reverse=True)#字典按照次数进行排序
#         print('res',res)
        top1=res[0][0]
        top2 = '_'.join([i[0] for i in res[:2]])
        top3 = '_'.join([i[0] for i in res[:3]])
        top4 = '_'.join([i[0] for i in res[:4]])
        top5='_'.join([i[0] for i in res[:5]])
        top10 = '_'.join([i[0] for i in res[:10]])
        return (top1,top2,top3,top4,top5,top10)
    org['top']=org['taglist'].apply(top)###运用top函数
    print('top finish')
    org['top1']=org['top'].apply(lambda x:x[0])
    org['top2'] = org['top'].apply(lambda x: x[1])
    org['top3'] = org['top'].apply(lambda x: x[2])
    org['top4'] = org['top'].apply(lambda x: x[3])
    org['top5'] = org['top'].apply(lambda x: x[4])
    org['top10'] = org['top'].apply(lambda x: x[5])
#     print(org)
#     org.to_csv('org.csv',index=False)
    return org[['user_id','top1','top2','top3','top4','top5','top10']]

In [24]:
user_taglist_new_feature= property_feature(user_taglist)

top finish


In [25]:
user_taglist_new_feature.sample(10)

,user_id,top1,top2,top3,top4,top5,top10
235899,138423,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
161370,462400,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_340,2017_2710_1804_1911_340_70_1876_727_2642_2300
126269,557342,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
569584,295002,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
150456,371297,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_631
285853,500436,2017,2017_2710,2017_2710_1654,2017_2710_1654_4185,2017_2710_1654_4185_421,2017_2710_1654_4185_421_2500_2540_2018_691_1812
377600,748629,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
384883,457925,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
339376,586287,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
410803,517797,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_440_70_2300_719_4185


In [26]:
import gc
del user_taglist_new_feature
del user_taglist['taglist']
del user_taglist['top']
gc.collect()

73

In [27]:
user_taglist.sample(10)

,user_id,insertdate,top1,top2,top3,top4,top5,top10
233788,5063,2018-05-29,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_5628_631
14944,626231,2018-07-28,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_440_5628_631_2642_2580
88070,427965,2018-12-10,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_440_70_1876_5628_631
425561,226528,2018-04-13,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
12084,719796,2018-05-25,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_70_1876_631_2642
471614,704891,2018-06-10,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_70_1478_2164_620_691
352413,902896,2018-10-14,2017,2017_2710,2017_2710_1654,2017_2710_1654_727,2017_2710_1654_727_2300,2017_2710_1654_727_2300_1178_2431_1964_1395_2245
444587,877086,2018-09-15,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
72084,220162,2018-08-22,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
260901,254259,2018-11-16,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_1876_631_2642_1100_421


In [28]:
#lablencoder
for each in ['top1','top2','top3','top4','top5','top10']:
  user_taglist[each]=LabelEncoder().fit_transform(user_taglist[each]).astype(np.uint32)

107

In [26]:
user_taglist.sample(10)

,user_id,insertdate,top1,top2,top3,top4,top5,top10
540182,459083,2018-11-19,18,146,538,1093,1606,2905
130422,794097,2018-12-03,18,166,687,1898,4308,38443
516696,221699,2018-10-01,18,166,687,1898,4308,38582
431996,51262,2018-04-25,18,166,687,1898,4308,38443
546230,791463,2018-06-21,18,166,687,1898,4308,39982
175985,293275,2018-05-27,18,166,687,1898,4308,38445
322076,837538,2018-11-07,18,166,687,1898,4308,37847
537118,115138,2018-12-11,18,166,687,1894,4238,22330
409934,284485,2018-12-21,18,166,687,1898,4308,37847
228941,389673,2018-11-26,18,149,590,1485,2916,11055


In [97]:
user_taglist.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615160 entries, 0 to 615159
Data columns (total 8 columns):
user_id       615160 non-null uint32
insertdate    615160 non-null datetime64[ns]
top1          615160 non-null uint32
top2          615160 non-null uint32
top3          615160 non-null uint32
top4          615160 non-null uint32
top5          615160 non-null uint32
top10         615160 non-null uint32
dtypes: datetime64[ns](1), uint32(7)
memory usage: 21.1 MB


In [100]:
user_taglist['top5'].nunique()

6268

In [70]:
user_behavior_logs = pd.read_csv(path+"user_behavior_logs.csv",encoding='utf8',parse_dates=['behavior_time'],
                                 dtype={'user_id':np.uint32,'behavior_type':np.uint32}) #1G

In [30]:
user_behavior_logs.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55781271 entries, 0 to 55781270
Data columns (total 3 columns):
user_id          uint32
behavior_time    datetime64[ns]
behavior_type    uint32
dtypes: datetime64[ns](1), uint32(2)
memory usage: 851.2 MB


In [44]:
user_repay_logs = pd.read_csv(path+"user_repay_logs.csv",encoding='utf8',
                              parse_dates=[ 'due_date', 'repay_date'],
                    dtype={'user_id':np.uint32, 'listing_id':np.uint32,'order_id':np.uint32,  
                                'due_amt':np.float32,'repay_amt':np.float32}) #900M

In [49]:
get_null_value(user_repay_logs)

字段名为： user_id    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： listing_id 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： order_id   缺失值数量: 0    缺失数量占比： 0.00%
字段名为： due_date   缺失值数量: 0    缺失数量占比： 0.00%
字段名为： due_amt    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： repay_date 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： repay_amt  缺失值数量: 0    缺失数量占比： 0.00%


,字段名为,缺失值数量,缺失数量占比
0,user_id,0,0.00%
1,listing_id,0,0.00%
2,order_id,0,0.00%
3,due_date,0,0.00%
4,due_amt,0,0.00%
5,repay_date,0,0.00%
6,repay_amt,0,0.00%


In [32]:
user_repay_logs.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18001297 entries, 0 to 18001296
Data columns (total 7 columns):
user_id       uint32
listing_id    uint32
order_id      uint32
due_date      datetime64[ns]
due_amt       float32
repay_date    datetime64[ns]
repay_amt     float32
dtypes: datetime64[ns](2), float32(2), uint32(3)
memory usage: 618.0 MB


In [50]:
user_repay_logs.sample(10)

,user_id,listing_id,order_id,due_date,due_amt,repay_date,repay_amt
14001225,2025,977650,6,2018-03-15,228.970596,2200-01-01,228.970596
6628634,868787,4554097,1,2018-10-29,296.860291,2018-10-20,296.860291
12814721,636345,1563008,2,2018-01-26,385.061890,2017-11-26,385.061890
6830823,196560,590473,2,2017-09-17,437.077301,2017-09-17,437.077301
7935467,395321,972751,2,2017-11-14,145.242401,2017-11-07,145.242401
13221335,221474,1267964,1,2017-11-24,577.238098,2017-11-22,577.238098
3298567,609625,2242508,12,2019-02-07,101.560699,2018-03-21,101.560699
7874435,135245,3478917,4,2018-09-21,244.347107,2018-09-21,244.347107
3996926,718498,2880461,3,2018-07-03,203.758408,2018-07-03,203.758408
803062,132568,2548914,4,2018-07-07,975.293091,2018-07-07,975.293091


In [0]:

# train = pd.read_csv(path+"train.csv",encoding='utf8',parse_dates=['auditing_date', 'due_date', 'repay_date'],
#                     dtype={'user_id':np.uint32, 'listing_id':np.uint32, 
#                                 #'due_amt':np.float32,'repay_amt':np.float32
#                                }
#                    ) #100W
# test = pd.read_csv(path+"test.csv",encoding='utf8',parse_dates=['auditing_date', 'due_date']) #13W
# submission = pd.read_csv(path+"/submission/submission.csv",encoding='utf8',
#                         dtype={'user_id':np.uint32, 'listing_id':np.uint32,'repay_date':str,'repay_amt':np.float32
#                                }) #398W
# listing_info = pd.read_csv(path+"listing_info.csv",encoding='utf8',
#                           dtype={'user_id':np.uint32, 'listing_id':np.uint32,'term':np.uint32,'rate':np.floate32,'principal':np.float32 
#                                 #'due_amt':np.float32,'repay_amt':np.float32
#                                })
# user_info = pd.read_csv(path+"user_info.csv",encoding='utf8',dtype={'user_id':np.uint32, 'age':np.uint32,
#                                })
# user_taglist = pd.read_csv(path+"user_taglist.csv",encoding='utf8',dtype={'user_id':np.uint32})
# user_behavior_logs = pd.read_csv(path+"user_behavior_logs.csv",encoding='utf8') #1G
# user_repay_logs = pd.read_csv(path+"user_repay_logs.csv",encoding='utf8') #900M


In [33]:

del user_repay_logs['user_id']
gc.collect()


35

In [34]:
listing_repay=pd.merge(listing_info,user_repay_logs,on=['listing_id'],how='left')

In [35]:

del listing_info
del user_repay_logs
gc.collect()

28

In [36]:
listing_repay.sample(10)

,user_id,listing_id,auditing_date,term,rate,principal,order_id,due_date,due_amt,repay_date,repay_amt
13334459,249030,294858,2017-05-16,12,9.3,850.0,11.0,2018-04-16,74.452003,2017-08-17,74.452003
9886884,543997,1324570,2017-10-31,9,7.6,4100.0,2.0,2017-12-31,470.105103,2017-12-31,470.105103
1935006,433587,859140,2017-08-28,6,8.6,1170.0,2.0,2017-10-28,199.922104,2017-08-30,199.922104
10553585,422582,4517370,2018-09-24,6,8.3,3430.0,1.0,2018-10-24,585.590515,2018-10-22,585.590515
16986109,383029,662009,2017-07-29,12,9.3,3960.0,9.0,2018-04-29,346.859009,2017-12-26,346.859009
13619999,445156,3452678,2018-05-19,9,7.2,1710.0,6.0,2018-11-19,195.745407,2018-09-22,195.745407
13625446,255543,1762945,2017-12-18,12,7.6,4450.0,10.0,2018-10-18,386.277802,2018-10-17,386.277802
8374616,150376,3572693,2018-05-29,9,7.2,7420.0,4.0,2018-09-29,849.375000,2018-09-29,849.375000
12641057,425829,2031108,2018-01-17,9,7.2,1400.0,5.0,2018-06-17,160.259399,2018-06-17,160.259399
3493910,612913,4985527,2018-11-26,6,7.2,6530.0,6.0,2019-05-26,1111.302246,2018-11-26,1111.302246


In [37]:
sy_listing_id=list(set(listing_repay['listing_id'])-set(train['listing_id']))

In [38]:
sy_listing_id[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [39]:
sy_listing_repay=listing_repay[listing_repay['listing_id'].isin(sy_listing_id)]

In [40]:
sy_listing_repay.sample(5)

,user_id,listing_id,auditing_date,term,rate,principal,order_id,due_date,due_amt,repay_date,repay_amt
3044011,633367,5116582,2018-12-13,9,7.2,1170.0,2.0,2019-02-13,133.931107,2019-01-04,133.931107
6347341,719339,2507396,2018-03-04,12,8.3,1500.0,6.0,2018-09-04,130.692001,2018-04-12,130.692001
17274279,170836,566532,2017-07-13,12,9.3,2800.0,5.0,2017-12-13,245.253799,2017-12-13,245.253799
7593597,269711,2097249,2018-01-24,6,7.2,1170.0,3.0,2018-04-24,199.115402,2018-04-20,199.115402
4001018,756175,3440350,2018-05-18,12,8.6,1220.0,7.0,2018-12-18,106.465500,2018-12-16,106.465500


In [41]:
for each in  ['user_id','auditing_date','due_date',	'due_amt',	'repay_date',	'repay_amt']:
    del listing_repay[each]
gc.collect()

55

In [42]:
train=pd.merge(train,listing_repay,on=['listing_id'],how='left')

In [43]:
del listing_repay
gc.collect()

35

In [120]:
train.sample(10)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id,term,rate,principal,order_id
15450,2018-07-30,2210.903809,2018-08-30,4098952,2210.903809,2018-08-28,133894,6,8.3,12950.0,NaN
600202,2018-03-14,824.191406,2018-04-14,2625136,0.000000,2080-01-01,763062,9,7.2,7200.0,NaN
1470579,2018-02-06,151.463806,2018-03-06,2234222,151.463806,2018-03-05,151596,6,7.2,890.0,4.0
190856,2018-06-04,109.892097,2018-07-04,3635664,109.892097,2018-06-15,383232,9,7.2,960.0,NaN
35158,2018-01-19,761.232300,2018-02-19,2047861,761.232300,2018-02-19,331376,9,7.2,6650.0,6.0
1436834,2018-12-07,111.036797,2019-01-07,5066742,111.036797,2018-12-23,253190,9,7.2,970.0,NaN
47262,2018-10-11,394.689301,2018-11-11,4630947,394.689301,2018-10-12,910954,3,7.2,1170.0,NaN
9068,2018-04-01,258.704498,2018-05-01,2847931,258.704498,2018-05-01,533099,9,7.2,2260.0,NaN
753398,2018-10-02,544.616272,2018-11-02,4574631,544.616272,2018-10-08,239575,6,8.3,3190.0,NaN
1067851,2018-03-11,571.438721,2018-04-11,2591047,571.438721,2018-04-11,437155,6,8.0,3350.0,2.0


In [44]:
train=train.append(sy_listing_repay)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [45]:
train.sample(20)

,auditing_date,due_amt,due_date,listing_id,order_id,principal,rate,repay_amt,repay_date,term,user_id
12965755,2018-01-07,517.359680,2018-06-07,1934027,5.0,3040.0,7.2,517.359680,2018-06-07,6,649285
11525981,2018-08-04,336.330414,2018-10-04,4133075,2.0,1970.0,8.3,336.330414,2018-10-04,6,273132
6539912,2017-09-05,145.242401,2017-11-05,912683,2.0,850.0,8.6,145.242401,2017-09-15,6,426497
13662143,2017-12-18,195.745407,2018-08-18,1752606,8.0,1710.0,7.2,195.745407,2018-02-27,9,728365
1568363,2019-03-10,125.683403,2019-04-10,5426113,NaN,740.0,6.5,0.000000,2080-01-01,6,340912
12573089,2018-06-18,218.340698,2018-07-18,3768749,1.0,1280.0,8.0,218.340698,2018-07-18,6,424039
11748869,2017-08-18,920.577820,2018-06-18,798215,10.0,10510.0,9.3,920.577820,2018-06-13,12,622855
15133870,2018-10-08,163.897095,2018-11-08,4612322,1.0,960.0,8.3,163.897095,2018-11-05,6,246098
19153987,2018-04-30,199.749603,2018-10-30,3231571,6.0,1170.0,8.3,199.749603,2018-10-29,6,844523
15615030,2018-09-12,414.864502,2018-10-12,4426217,1.0,2430.0,8.3,414.864502,2018-10-12,6,205726


In [46]:
train=pd.merge(train,user_info,on=['user_id'],how='left')

In [47]:
del user_info
gc.collect()

38

In [48]:
train['order_id']=train['order_id'].astype(np.float32)

In [127]:
train.sample(15)

,auditing_date,due_amt,due_date,listing_id,order_id,principal,rate,repay_amt,repay_date,term,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate
16000905,2018-04-22,578.761719,2018-08-22,3124854,4.0,3390.0,8.3,578.761719,2200-01-01,6,623092,2017-08-01,1,31,4,11,11077,2018-11-02
15350828,2017-12-13,205.725494,2018-04-13,1720097,4.0,2370.0,7.6,205.725494,2018-03-15,12,469341,2017-04-01,0,38,4,4,4206,2018-03-15
16659530,2018-10-08,155.360596,2018-12-08,4610306,2.0,910.0,8.3,155.360596,2018-12-08,6,819229,2018-03-01,0,31,12,4,4348,2019-03-03
3339821,2018-03-05,963.635620,2018-05-05,2520263,2.0,11060.0,8.3,963.635620,2018-05-05,12,150093,2016-04-01,1,32,8,8,8256,2018-06-05
10707829,2017-12-11,252.599701,2018-01-11,1699279,1.0,2910.0,7.6,252.599701,2018-01-05,12,714026,2017-11-01,1,27,22,12,12190,2018-06-14
11293297,2017-11-25,660.440613,2018-02-25,1555619,3.0,5760.0,7.6,660.440613,2018-02-21,9,142981,2016-04-01,1,31,25,25,25140,2018-05-27
10539521,2017-12-17,141.944000,2018-07-17,1744515,7.0,1240.0,7.2,141.944000,2018-07-17,9,309117,2016-11-01,1,29,8,31,31067,2018-07-23
12131758,2017-10-18,852.893616,2018-02-18,1215351,4.0,5000.0,8.0,852.893616,2018-02-18,6,234150,2016-08-01,0,32,8,8,8343,2018-07-09
8995849,2018-07-05,249.260101,2018-12-05,3919361,5.0,1460.0,8.3,249.260101,2018-12-05,6,623933,2017-08-01,1,33,26,4,4155,2019-02-27
3554193,2017-09-06,946.639893,2018-03-06,915087,6.0,5540.0,8.6,946.639893,2018-03-06,6,102853,2016-01-01,1,27,2,17,17096,2018-03-19


In [129]:
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20711267 entries, 0 to 20711266
Data columns (total 18 columns):
auditing_date    datetime64[ns]
due_amt          float32
due_date         datetime64[ns]
listing_id       uint32
order_id         float32
principal        float32
rate             float32
repay_amt        float32
repay_date       datetime64[ns]
term             uint32
user_id          uint32
reg_mon          datetime64[ns]
gender           uint32
age              uint32
cell_province    uint32
id_province      uint32
id_city          uint32
insertdate       datetime64[ns]
dtypes: datetime64[ns](5), float32(5), uint32(8)
memory usage: 1.9 GB


In [49]:

user_taglist.rename(columns={'insertdate': 'tag_insert_date'}, inplace=True)
user_taglist = user_taglist.sort_values(by='tag_insert_date', ascending=False).drop_duplicates('user_id').reset_index(drop=True)


In [50]:
train=pd.merge(train,user_taglist,on=['user_id'],how='left')

In [51]:
del user_taglist
gc.collect()

42

In [52]:
train.head()

,auditing_date,due_amt,due_date,listing_id,order_id,principal,rate,repay_amt,repay_date,term,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate,tag_insert_date,top1,top2,top3,top4,top5,top10
0,2018-04-25,72.116699,2018-05-25,3163926,5.0,630.0,7.2,72.116699,2018-05-25,9,748147,2017-12-01,1,21,20,20,20089,2018-04-24,2018-11-12,18.0,166.0,687.0,1898.0,4308.0,38266.0
1,2018-04-25,72.116699,2018-05-25,3163926,4.0,630.0,7.2,72.116699,2018-05-25,9,748147,2017-12-01,1,21,20,20,20089,2018-04-24,2018-11-12,18.0,166.0,687.0,1898.0,4308.0,38266.0
2,2018-04-25,72.116699,2018-05-25,3163926,1.0,630.0,7.2,72.116699,2018-05-25,9,748147,2017-12-01,1,21,20,20,20089,2018-04-24,2018-11-12,18.0,166.0,687.0,1898.0,4308.0,38266.0
3,2018-04-25,72.116699,2018-05-25,3163926,3.0,630.0,7.2,72.116699,2018-05-25,9,748147,2017-12-01,1,21,20,20,20089,2018-04-24,2018-11-12,18.0,166.0,687.0,1898.0,4308.0,38266.0
4,2018-04-25,72.116699,2018-05-25,3163926,2.0,630.0,7.2,72.116699,2018-05-25,9,748147,2017-12-01,1,21,20,20,20089,2018-04-24,2018-11-12,18.0,166.0,687.0,1898.0,4308.0,38266.0


In [53]:
train.to_parquet('train.parquet.gzip',compression='gzip',index=False)

In [55]:
ls -lh

total 867M
drwxr-xr-x 3 root root 4.0K Jun 13 08:49 data/
-rw-r--r-- 1 1000 1000 646M Jul  1 08:27 data.rar
-rw-r--r-- 1 root root  977 Jul  2 01:16 df.parquet.gzip
-rw-r--r-- 1 1000 1000 313K Jul  2 01:20 magic_competiton_eda.ipynb
-rw-r--r-- 1 root root  28K Jul  1 09:29 magic_mirror_eda.ipynb
-rw-r--r-- 1 root root 3.7K Jul  2 01:18 temp.ipynb
-rw-r--r-- 1 root root 221M Jul  2 01:20 train.parquet.gzip


In [133]:
train.sample(10)

,auditing_date,due_amt,due_date,listing_id,order_id,principal,rate,repay_amt,repay_date,term,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate,tag_insert_date,top1,top2,top3,top4,top5,top10
3679686,2018-02-22,155.226395,2018-03-22,2389688,1.0,910.0,8.0,155.226395,2018-03-22,6,700474,2017-10-01,1,27,29,29,29091,2018-05-12,2018-05-12,18.0,166.0,687.0,1898.0,4308.0,38533.0
14837843,2017-05-22,102.481003,2018-03-22,317364,10.0,1170.0,9.3,102.481003,2018-03-20,12,510335,2017-05-01,1,25,20,6,6281,2018-08-21,2018-08-21,18.0,166.0,687.0,1898.0,4308.0,37875.0
15142666,2017-12-20,114.471001,2018-01-20,1777977,1.0,1000.0,7.2,114.471001,2018-01-20,9,357423,2016-12-01,0,32,16,16,16146,2018-01-26,2018-09-24,18.0,166.0,687.0,1898.0,4308.0,38443.0
15960368,2018-04-26,102.102203,2018-08-26,3178847,4.0,1170.0,8.6,102.102203,2018-08-14,12,739837,2017-11-01,1,25,26,26,26057,2019-02-01,2019-02-01,18.0,166.0,687.0,1898.0,4308.0,38443.0
15670494,2017-12-03,190.335205,2018-06-03,1628798,6.0,1660.0,7.6,190.335205,2018-04-27,9,413297,2017-02-01,1,32,28,28,28340,2018-07-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN
8347123,2018-02-23,327.511200,2018-03-23,2395975,1.0,1920.0,8.0,327.511200,2018-03-15,6,718909,2017-11-01,1,28,4,4,4152,2018-06-06,NaT,NaN,NaN,NaN,NaN,NaN,NaN
5181382,2018-01-05,144.656403,2018-02-05,1914562,1.0,850.0,7.2,144.656403,2018-01-06,6,525053,2017-05-01,1,25,17,17,17242,2018-03-15,2018-10-08,18.0,166.0,687.0,1898.0,4308.0,43781.0
14591,2018-12-15,942.004089,2019-01-15,5127209,NaN,5540.0,6.9,942.004089,2019-01-15,6,9099,2014-08-01,1,28,26,26,26131,2018-12-14,2018-12-14,18.0,166.0,687.0,1894.0,4238.0,22312.0
20259857,2017-12-07,97.300301,2018-05-07,1663522,5.0,850.0,7.2,97.300301,2018-01-11,9,418130,2017-03-01,1,25,5,5,5021,2018-06-27,2018-08-16,18.0,166.0,687.0,1898.0,4308.0,41759.0
14039543,2018-01-23,884.957275,2018-06-23,2088522,5.0,5200.0,7.2,884.957275,2018-06-16,6,473343,2017-04-01,1,48,17,17,17290,2018-08-29,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# user_taglist.rename(columns={'insertdate': 'tag_insert_date'}, inplace=True)
# user_taglist = user_taglist.sort_values(by='tag_insert_date', ascending=False).drop_duplicates('user_id').reset_index(drop=True)


In [137]:
user_behavior_logs.sample(10)

,user_id,behavior_time,behavior_type
5550251,142601,2018-02-05 17:41:52,1
41413923,868500,2018-10-21 21:02:45,1
28869041,165760,2018-11-25 16:04:36,1
28530411,8611,2017-10-20 18:14:33,3
14039644,462882,2018-07-14 13:25:55,1
47894960,647816,2018-07-15 13:57:11,1
18546483,531671,2017-11-29 18:29:31,1
17664087,628204,2018-03-17 11:13:21,1
11757713,869755,2018-06-15 20:01:04,3
26668441,695878,2018-08-20 18:28:46,3


In [73]:
user_behavior_logs_times=user_behavior_logs.groupby(['user_id','behavior_time','behavior_type']).size().reset_index()

In [75]:
user_behavior_logs_times.sample(50)

,user_id,behavior_time,behavior_type,0
13255190,246319,2018-09-27 15:39:08,1,1
42507869,861147,2018-10-25 18:17:39,1,1
32983101,636255,2017-09-07 16:12:15,3,1
11663554,217360,2019-01-05 09:29:21,1,1
20061566,374713,2018-11-08 14:50:47,1,1
29640471,565460,2018-05-06 23:07:53,1,1
36626637,715474,2018-09-25 16:19:10,1,1
42620298,864450,2018-09-13 23:05:11,1,1
21671743,406187,2018-04-13 09:04:22,1,1
31735680,609499,2018-01-04 23:44:03,1,1


In [76]:

user_behavior_logs_times.rename(columns={0:'group_size'},inplace=True)##要加columns=否则不会重命名变量


In [77]:
##合并之前先删除重复的记录
user_behavior_logs_times.shape

(44546291, 4)

In [65]:
# user_behavior_logs_times.drop_duplicates(inplace=True)##这个是groupby之后的是没有重复的

In [66]:
user_behavior_logs_times.shape

(25382008, 3)

In [79]:
user_behavior_logs_times[user_behavior_logs_times['behavior_time']==pd.to_datetime('2018-09-13 23:17:21')]

,user_id,behavior_time,behavior_type,group_size
41390088,829786,2018-09-13 23:17:21,1,1
41845090,842439,2018-09-13 23:17:21,3,10


In [80]:
print(user_behavior_logs.shape)
user_behavior_logs.drop_duplicates(inplace=True)
print(user_behavior_logs.shape)

(55781271, 3)
(44546291, 3)


In [81]:
user_behavior_logs=pd.merge(user_behavior_logs,user_behavior_logs_times,on=['user_id', 'behavior_time', 'behavior_type'],how='left')
#很占内存

In [82]:
user_behavior_logs.sample(10)

,user_id,behavior_time,behavior_type,group_size
8680010,48379,2017-08-23 13:38:48,1,1
20233112,277524,2018-03-14 13:28:53,1,1
32733368,102690,2018-05-25 13:28:42,1,1
27510346,320245,2018-04-19 14:35:14,1,1
39594760,417853,2018-03-16 18:08:26,1,2
11895303,300944,2018-04-07 11:03:55,1,1
38117201,351537,2018-02-21 16:32:06,1,1
31521312,736829,2018-01-03 15:22:25,1,1
34383309,902982,2018-10-02 21:26:06,1,1
20084315,261155,2018-10-28 16:29:58,1,1


In [83]:
user_behavior_logs.shape

(44546291, 4)

In [84]:
user_behavior_logs['user_id'].nunique()

920161

In [86]:
user_behavior_logs.to_parquet('user_behavior_logs.parquet.gzip',compression='gzip')

In [88]:
ls -lh

total 1.2G
drwxr-xr-x 3 root root 4.0K Jun 13 08:49 data/
-rw-r--r-- 1 1000 1000 646M Jul  1 08:27 data.rar
-rw-r--r-- 1 root root  977 Jul  2 01:16 df.parquet.gzip
-rw-r--r-- 1 1000 1000 338K Jul  2 02:42 magic_competiton_eda.ipynb
-rw-r--r-- 1 root root  28K Jul  1 09:29 magic_mirror_eda.ipynb
-rw-r--r-- 1 root root 9.6K Jul  2 02:12 temp.ipynb
-rw-r--r-- 1 root root 221M Jul  2 01:20 train.parquet.gzip
-rw-r--r-- 1 root root 350M Jul  2 02:42 user_behavior_logs.parquet.gzip


In [89]:
train.columns#insertdate是user_info的

Index(['auditing_date', 'due_amt', 'due_date', 'listing_id', 'order_id',
       'principal', 'rate', 'repay_amt', 'repay_date', 'term', 'user_id',
       'reg_mon', 'gender', 'age', 'cell_province', 'id_province', 'id_city',
       'insertdate', 'tag_insert_date', 'top1', 'top2', 'top3', 'top4', 'top5',
       'top10'],
      dtype='object')

In [97]:
ls

data/            magic_competiton_eda.ipynb  train.parquet.gzip
data.rar         magic_mirror_eda.ipynb      user_behavior_logs.parquet.gzip
df.parquet.gzip  temp.ipynb


In [98]:
train=pd.read_parquet('train.parquet.gzip')

In [100]:
train.loc[(pd.to_datetime(train['auditing_date'])>=pd.to_datetime(train['insertdate']))
          &
          (pd.to_datetime(train['auditing_date'])>=pd.to_datetime(train['tag_insert_date']))].shape
#          ,['auditing_date','due_amt','insertdate','tag_insert_date']].head(20)

(778668, 25)

In [99]:
train.shape

(20711267, 25)

In [0]:
#################################################################################

In [85]:
train.shape

(20711267, 25)

In [0]:
all_data[all_data['repay_amt'].isnull()].shape

(130000, 7)

In [0]:
all_data.head()

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id
0,2018-04-25,72.1167,2018-05-25,3163926,72.1167,2018-05-25,748147
1,2018-06-09,258.7045,2018-07-09,3698760,258.7045,2018-07-08,672952
2,2018-02-18,307.9270,2018-03-18,2355665,\N,\N,404196
3,2018-01-13,252.9809,2018-02-13,1994522,252.9809,2018-02-13,342769
4,2018-06-01,107.6503,2018-07-01,3602352,107.6503,2018-06-25,828139


In [0]:
import gc
del listing_info['user_id'], listing_info['auditing_date']
gc.collect()

14

In [0]:
all_data=pd.merge(all_data,listing_info,on=['listing_id'],how='left')##只将all_data中的listing_id添加了一些维度，对于新增的listing_id并没有添加

In [0]:
all_data.sample(20)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id,term,rate,principal
112683,2018-11-19,121.3008,2018-12-19,4943841,121.3008,2018-12-18,833995,12,8.6,1390
769391,2018-11-21,326.3780,2018-12-21,4957486,326.378,2018-12-21,80274,12,8.6,3740
843706,2018-05-17,302.4463,2018-06-17,3427903,302.4463,2018-06-15,342946,6,8.6,1770
147577,2018-11-13,299.3252,2018-12-13,4904765,299.3252,2018-12-13,11206,12,8.6,3430
737902,2018-08-15,245.8456,2018-09-15,4209031,245.8456,2018-09-14,806539,6,8.3,1440
1066681,2019-03-09,383.0054,2019-04-09,5425226,NaN,NaN,564547,9,6.9,3350
534145,2018-03-19,1277.4966,2018-04-19,2688511,1277.4966,2018-03-26,234760,9,7.2,11160
962549,2018-09-25,478.0331,2018-10-25,4526515,478.0331,2018-10-25,139634,6,8.3,2800
719183,2018-02-25,394.0369,2018-03-25,2415975,394.0369,2018-03-19,627467,6,8.0,2310
139997,2018-04-26,1891.6457,2018-05-26,3179597,1891.6457,2018-05-26,160883,6,8.3,11080


In [0]:
all_data.shape

(1130000, 10)

In [0]:
user_info.head()

,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate
0,483833,2017-04,男,19,c29,c26,c26241,2018-12-11
1,156772,2016-05,男,31,c11,c11,c11159,2018-02-13
2,173388,2016-05,男,34,c02,c02,c02182,2018-08-21
3,199107,2016-07,女,25,c09,c09,c09046,2018-06-05
4,122560,2016-03,男,23,c05,c05,c05193,2018-04-02


In [0]:
## 表中有少数user不止一条记录，因此按日期排序，去重，只保留最新的一条记录。
user_info = user_info.sort_values(by='insertdate', ascending=False).drop_duplicates('user_id').reset_index(drop=True)

In [0]:
all_data=pd.merge(all_data,user_info,on=['user_id'],how='left')

In [0]:
all_data.sample(10)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id,term,rate,principal,reg_mon,gender,age,cell_province,id_province,id_city,insertdate
885066,2018-10-08,945.8228,2018-11-08,4612184,945.8228,2018-11-07,116374,6,8.3,5540,2016-02,女,24,c04,c21,c21299,2018-10-07
323,2018-06-21,131.6416,2018-07-21,3790584,131.6416,2018-07-21,719769,9,7.2,1150,2017-11,女,42,c24,c24,c24044,2018-06-20
393581,2018-09-08,72.4314,2018-10-08,4391798,72.4314,2018-09-21,268513,12,8.6,830,2016-09,女,34,c16,c16,c16146,2018-02-26
436420,2018-03-15,525.3824,2018-04-15,2641871,525.3824,2018-04-13,65580,6,8.0,3080,2015-10,男,27,c31,c31,c31114,2018-02-05
535421,2018-10-16,394.6893,2018-11-16,4668817,394.6893,2018-11-16,745574,3,7.2,1170,2017-12,女,25,c05,c05,c05021,2018-10-15
756932,2018-10-18,571.4387,2018-11-18,4683432,571.4387,2018-11-18,284529,6,8.0,3350,2016-10,男,25,c04,c04,c04331,2018-10-17
650945,2018-10-22,327.2507,2018-11-22,4714983,327.2507,2018-11-22,666661,12,8.6,3750,2017-09,男,26,c04,c04,c04264,2018-10-21
469354,2018-03-11,133.9311,2018-04-11,2593135,133.9311,2018-04-11,209081,9,7.2,1170,2016-07,男,26,c09,c09,c09167,2018-03-10
169179,2018-05-01,452.1605,2018-06-01,3233494,\N,\N,323739,9,7.2,3950,2016-11,男,32,c22,c22,c22308,2018-04-30
796803,2018-04-14,766.5604,2018-05-14,3029515,766.5604,2018-05-13,482494,6,8.3,4490,2017-04,男,27,c05,c05,c05296,2018-04-13


In [0]:
user_taglist.rename(columns={'insertdate': 'tag_insert_date'}, inplace=True)
user_taglist = user_taglist.sort_values(by='tag_insert_date', ascending=False).drop_duplicates('user_id').reset_index(drop=True)


In [0]:
all_data=pd.merge(all_data,user_taglist,on=['user_id'],how='left')

In [0]:
all_data.sample(10)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id,term,rate,principal,reg_mon,gender,age,cell_province,id_province,id_city,insertdate,taglist,tag_insert_date
1022751,2019-02-15,202.6137,2019-03-15,5336367,NaN,NaN,88883,9,7.2,1770,2015-12,男,31,c31,c31,c31255,2019-02-14,5906|719|5865|3369|4453|3006|70|4524|5723|4787...,2019-02-14
353326,2018-12-12,384.2833,2019-01-12,5107265,384.2833,2019-01-12,54469,6,6.9,2260,2015-09,女,34,c20,c19,c19239,2018-12-11,NaN,NaN
140123,2018-03-25,161.4041,2018-04-25,2757432,161.4041,2018-04-24,482680,9,7.2,1410,2017-04,男,37,c19,c19,c19104,2018-03-24,2710|1228|1541|2901|1402|2500|877|4592|1804|84...,2018-03-24
316259,2018-03-06,171.7065,2018-04-06,2536224,171.7065,2018-04-03,545759,9,7.2,1500,2017-06,女,35,c04,c04,c04344,2018-03-05,3017|1832|1204|1368|354|3695|3691|2648|1911|27...,2018-03-05
832785,2018-04-01,371.8615,2018-05-01,2848283,371.8615,2018-04-19,539922,6,8.0,2180,2017-06,男,24,c05,c05,c05296,2018-03-31,230|2242|1478|2164|2250|3324|1876|3781|1059|83...,2018-03-31
278046,2018-04-01,136.2205,2018-05-01,2845894,136.2205,2018-04-28,316148,9,7.2,1190,2016-11,男,22,c26,c26,c26241,2018-03-31,NaN,NaN
619391,2018-02-07,352.5707,2018-03-07,2242111,352.5707,2018-03-06,693139,9,7.2,3080,2017-10,男,20,c12,c12,c12007,2018-02-06,2134|3088|2253|4864|4749|1541|5320|719|2017|20...,2018-02-06
1070322,2019-03-20,198.7159,2019-04-20,5457877,NaN,NaN,686712,6,6.5,1170,2017-10,男,22,c23,c26,c26047,2019-03-19,60|2580|127|2710|3151|4219|2642|155|1911|4563|...,2019-03-19
1114162,2019-02-08,176.8383,2019-03-08,5311810,NaN,NaN,484890,6,6.9,1040,2017-04,女,27,c04,c04,c04251,2019-02-07,NaN,NaN
827317,2018-03-29,144.9917,2018-04-29,2814899,\N,\N,479372,6,8.0,850,2017-04,男,22,c21,c21,c21240,2018-03-28,2245|2431|943|1285|2649|1964|3580|124|1567|620...,2018-03-28


In [0]:
all_data=pd.merge(all_data,user_repay_logs[['listing_id','order_id']],on=['listing_id'],how='left')

In [0]:
all_data.sample(5)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id,term,rate,principal,reg_mon,gender,age,cell_province,id_province,id_city,insertdate,taglist,tag_insert_date,order_id
791713,2018-03-07,97.3003,2018-04-07,2546932,\N,\N,609863,9,7.2,850,2017-08,男,21,c29,c29,c29091,2018-03-06,3527|1826|2739|121|620|2431|1950|1823|676|1876...,2018-03-06,NaN
263385,2018-04-19,861.9668,2018-05-19,3090846,861.9668,2018-05-17,624811,9,7.2,7530,2017-08,男,26,c19,c19,c19335,2018-04-18,NaN,NaN,5.0
1449235,2018-04-23,2029.5713,2018-05-23,3144913,\N,\N,216253,9,7.2,17730,2016-07,男,31,c01,c29,c29128,2018-04-22,943|2794|645|449|445|2132|675|4224|3508|3972|4...,2018-11-25,2.0
109036,2018-11-03,124.7915,2018-12-03,4829996,124.7915,2018-12-01,655643,12,8.6,1430,2017-09,男,28,c02,c02,c02163,2018-03-04,1803|2580|1402|1029|5071|4770|1259|1787|2631|1...,2018-11-02,NaN
778126,2018-03-17,227.4265,2018-04-17,2662575,227.4265,2018-04-17,191284,12,7.6,2620,2016-06,女,25,c23,c23,c23141,2018-03-16,NaN,NaN,5.0


In [0]:
all_data.to_csv('all_data.csv',index=False)

In [0]:
train.loc[(train['auditing_date']>=pd.to_datetime('2018-01-05'))&(train['auditing_date']<=pd.to_datetime('2018-08-05'))].head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
0,748147,3163926,2018-04-25,2018-05-25,72.1167,2018-05-25,72.1167
1,672952,3698760,2018-06-09,2018-07-09,258.7045,2018-07-08,258.7045
2,404196,2355665,2018-02-18,2018-03-18,307.9270,\N,\N
3,342769,1994522,2018-01-13,2018-02-13,252.9809,2018-02-13,252.9809
4,828139,3602352,2018-06-01,2018-07-01,107.6503,2018-06-25,107.6503


In [0]:
print(train.describe())

              user_id    listing_id         due_amt
count  1000000.000000  1.000000e+06  1000000.000000
mean    446177.855141  3.797037e+06      452.026625
std     257534.686801  9.424460e+05      514.202155
min          1.000000  1.873193e+06       17.453300
25%     225120.750000  3.046201e+06      160.482400
50%     444321.000000  3.832456e+06      286.740100
75%     664508.250000  4.657050e+06      546.026800
max     924400.000000  5.207722e+06    18827.018400


In [0]:
print(listing_info.shape)

(5484891, 6)


In [0]:
print(listing_info['listing_id'].nunique())

5484891


In [0]:
# print(listing_info['user_id'].nunique())
# print((train['user_id'].append(test['user_id']).nunique())

set(listing_info['user_id'])-set(train['user_id'])-set(test['user_id'])

set()

In [0]:
train[train['user_id']==65747]	##train中的标的只是用户的部分标的情况，还有一些标的没有给出因为是抽样的把，但是listing_info中的标的有更多，包括不在train中的

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
84998,65747,1879801,2018-01-01,2018-02-01,437.2793,2018-01-23,437.2793


In [0]:
len(set(listing_info['listing_id'])-set(train['listing_id'])-set(test['listing_id']))
##4354891这么多的标的既不在train中也不在test中，但是若统计一段时间内用户的借款次数要把这些加上；预测的用户部分标的的还款时间
##也就是统计用户信息，以user_id为键，（或者user_id listing_id）

4354891

In [0]:
user_repay_logs[user_repay_logs['listing_id'].isin([1,2,3,4])]

,user_id,listing_id,order_id,due_date,due_amt,repay_date,repay_amt
234684,65747,2,12,2017-07-05,323.2095,2017-05-15,323.2095
2817251,133141,4,12,2017-07-06,55.1821,2017-01-13,55.1821
17182239,194014,1,12,2017-07-05,389.7784,2017-07-03,389.7784
17205590,4548,3,12,2017-07-06,973.1321,2017-07-06,973.1321


In [0]:
user_repay_logs[user_repay_logs['listing_id'].isin([5138741,3661187])]

,user_id,listing_id,order_id,due_date,due_amt,repay_date,repay_amt
10188421,3,3661187,4,2018-10-06,509.396,2018-10-06,509.396
10188422,3,3661187,5,2018-11-06,509.396,2018-11-06,509.396
10188423,3,3661187,6,2018-12-06,509.396,2018-12-06,509.396
10188424,3,3661187,2,2018-08-06,509.396,2018-08-06,509.396
10188425,3,3661187,1,2018-07-06,509.396,2018-07-06,509.396
10188426,3,3661187,3,2018-09-06,509.396,2200-01-01,509.396


In [0]:
print(train.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 7 columns):
user_id          1000000 non-null int64
listing_id       1000000 non-null int64
auditing_date    1000000 non-null datetime64[ns]
due_date         1000000 non-null datetime64[ns]
due_amt          1000000 non-null float64
repay_date       1000000 non-null object
repay_amt        1000000 non-null object
dtypes: datetime64[ns](2), float64(1), int64(2), object(2)
memory usage: 162.4 MB
None


In [0]:
print((train['due_date']-train['auditing_date']).dt.days.head())

NameError: ignored

In [0]:
train['user_id'].nunique()/train['user_id'].shape[0]

0.823732

In [0]:
print(train.shape)
print(test.shape)

(1000000, 7)
(130000, 5)


In [0]:
print(train['user_id'].nunique())

823732


In [0]:
print(train['listing_id'].nunique())##说明不同的用户，有相同的标的，

1000000


In [0]:
print(test['user_id'].nunique())


130000


In [0]:
sort_test=test.sort_values(by='user_id')

In [0]:
print(sort_test.head(10))

        user_id  listing_id auditing_date    due_date    due_amt
26177         5     5348580    2019-02-18  2019-03-18   205.7445
95146         8     5288930    2019-02-02  2019-03-02   521.9878
7011         20     5335952    2019-02-15  2019-03-15    88.1426
117274       22     5382105    2019-02-26  2019-03-26   125.7629
55784        24     5433181    2019-03-12  2019-04-12   560.4247
18638        29     5422932    2019-03-09  2019-04-09   467.6095
84268        35     5414862    2019-03-07  2019-04-07   226.9417
25043        41     5462399    2019-03-22  2019-04-22  1450.8678
34878        46     5480102    2019-03-29  2019-04-29   220.6568
35276        55     5424428    2019-03-09  2019-04-09   290.1735


In [0]:
print(sort_test.shape)

(130000, 5)


In [0]:
print(sort_test['user_id'].nunique())

130000


In [0]:
sort_info=listing_info.sort_values(by='listing_id')

In [0]:
print(sort_info.head())

         user_id  listing_id auditing_date  term  rate  principal
1341641   194014           1    2016-07-05    12   9.3       4450
1341640    65747           2    2016-07-05    12   9.3       3690
2715804     4548           3    2016-07-06    12   9.3      11110
1341639   133141           4    2016-07-06    12   9.3        630
4089125   198392           5    2016-07-07    12   9.3       2800


In [0]:
sort_info_user=listing_info.sort_values(by='user_id')

In [0]:
print(sort_info_user.head(10))#同一个用户在不同时间有不同的成交标的

         user_id  listing_id auditing_date  term  rate  principal
4307755        1     5138741    2018-12-17     9   7.2       5540
5016150        1      445999    2017-06-21    12   9.3       1380
2603034        2     1655550    2017-12-06    12   8.0       2800
1610929        2     4593188    2018-10-05     6   8.3       5430
623555         3      806012    2017-08-19    12   9.3       9740
928897         3     3661187    2018-06-06     9   7.2       4450
957421         3     1917365    2018-01-05    12   7.6       2800
50605          3     1353129    2017-11-03    12   8.6       8350
410070         3     4373926    2018-09-06     6   8.3       5320
898715         3     5079125    2018-12-08     9   7.2       5540


In [0]:
sort_train=train.sort_values(by='user_id')
#同一个人在不同时间有不同的成交标的##用户的成交次数作为特征
#借款用户可能有多个标的记录，但在测试集时间范围内每位用户只有一条记录。

In [0]:
print(sort_train.head(20))

        user_id  listing_id auditing_date    due_date    due_amt  repay_date  \
716258        1     5138741    2018-12-17  2019-01-17   634.1695  2019-01-11   
699724        2     4593188    2018-10-05  2018-11-05   927.0431  2018-11-04   
612838        3     3661187    2018-06-06  2018-07-06   509.3960  2018-07-06   
723079        3     5079125    2018-12-08  2019-01-08   634.1695  2019-01-08   
347041        4     1998686    2018-01-14  2018-02-14  1013.0686  2018-02-11   
5856          6     2896524    2018-04-04  2018-05-04   412.8005  2018-05-01   
81254         7     3737371    2018-06-13  2018-07-13  4036.6015  2018-07-13   
753040        8     4036140    2018-07-19  2018-08-19   283.4052          \N   
726131        9     4853712    2018-11-06  2018-12-06  1269.3415  2018-12-05   
887121        9     4777798    2018-10-28  2018-11-28   292.3439  2018-11-26   
150732       10     3225231    2018-04-30  2018-05-30   626.5648  2018-05-30   
946370       11     3380646    2018-05-1

In [0]:
sort_train_by_listid=train.sort_values(by='listing_id')

In [0]:
print(sort_train_by_listid.head(20))

        user_id  listing_id auditing_date    due_date   due_amt  repay_date  \
395543   438081     1873193    2018-01-01  2018-02-01   97.3003  2018-01-28   
864992   679097     1873199    2018-01-01  2018-02-01  200.8172  2018-01-04   
404472   633241     1873200    2018-01-01  2018-02-01  127.0628          \N   
271760   498924     1873205    2018-01-01  2018-02-01  226.6526  2018-02-01   
439912   342463     1873207    2018-01-01  2018-02-01  125.9181  2018-02-01   
466469   297792     1873212    2018-01-01  2018-02-01  163.3767  2018-01-31   
237865   711380     1873215    2018-01-01  2018-02-01  133.9311  2018-01-26   
639101   276089     1873220    2018-01-01  2018-02-01  267.8622  2018-01-30   
582993     8808     1873221    2018-01-01  2018-02-01  169.4171  2018-02-01   
393029   137332     1873223    2018-01-01  2018-02-01  116.7604  2018-01-30   
916595    84259     1873224    2018-01-01  2018-02-01  383.4779  2018-01-31   
475342   499298     1873232    2018-01-01  2018-02-0

In [0]:
sort_user_info=user_info.sort_values(by='user_id')

In [0]:
print(sort_user_info.head(10))#用户信息可能发生变更，表中同一用户可能存在多条数据  ##记录变更次数作为特征

        user_id  reg_mon gender  age cell_province id_province id_city  \
423710        1  2008-03      男   35           c22         c31  c31067   
468220        2  2008-03      男   46           c09         c09  c09205   
245236        3  2008-03      男   35           c22         c05  c05126   
233132        4  2008-04      男   36           c17         c17  c17246   
953223        5  2008-04      男   45           c04         c27  c27120   
611348        6  2008-04      男   36           c14         c14  c14027   
783710        7  2008-04      男   38           c05         c05  c05021   
241066        8  2008-04      男   36           c04         c04  c04152   
243463        8  2008-04      男   35           c04         c04  c04152   
815086        9  2008-04      男   34           c05         c05  c05169   

        insertdate  
423710  2018-12-16  
468220  2018-10-04  
245236  2018-06-05  
233132  2018-01-13  
953223  2019-02-17  
611348  2018-04-03  
783710  2018-06-12  
241066  2019-02-0

In [0]:
sort_user_taglist=user_taglist.sort_values(by='user_id')

In [0]:
print(sort_user_taglist.head(20))#用户标签可能发生变更，表中同一用户可能存在多条数据；若在表中无法查到用户标签信息，则表示该用户标签信息未知。

        user_id                                            taglist  insertdate
88108         2  2017|4853|3345|1401|320|727|3703|554|1804|2139...  2018-10-04
321635        4  323|683|2250|436|1288|1044|1876|181|357|50|365...  2018-01-13
221336        5  1868|2580|483|1303|904|3604|4028|2049|196|3551...  2019-02-17
133862        7  2005|271|2365|2216|933|1804|1788|719|2572|469|...  2018-06-12
502184        9  2710|1911|3504|3574|864|4688|127|2642|2127|420...  2018-10-27
501338        9  5852|3703|287|5762|5539|5465|4207|3574|3519|11...  2018-11-05
386047       10  1183|2492|2049|472|176|592|4703|266|595|721|31...  2018-04-29
429398       12  2017|1804|883|631|2365|552|1178|970|1952|2436|...  2018-03-28
273693       13  2809|638|1820|550|20|271|4938|2026|1697|1395|1...  2018-03-04
570753       15  2047|141|1863|325|4185|58|1127|4852|4766|620|4...  2018-03-03
330097       17  411|1255|5539|287|3017|1280|127|305|418|3519|1...  2018-11-27
116627       18  2018|1729|3844|2443|1979|3219|1769|

In [0]:
sort_user_behavior_logs=user_behavior_logs.sort_values(by='user_id')

In [0]:
print(sort_user_behavior_logs.shape)

(55781271, 3)


In [0]:
print(sort_user_behavior_logs['user_id'].nunique())

920161


In [0]:
print(sort_user_behavior_logs.head(20))#相同用户的相同行为在同一秒内可能出现多条数据。###这样的数据要去重

          user_id        behavior_time  behavior_type
36614258        1  2018-08-11 20:37:46              1
36535106        1  2018-07-04 17:29:50              3
36535100        1  2018-07-04 17:29:47              3
36535096        1  2018-07-04 17:29:47              3
36535094        1  2018-07-04 17:29:47              3
36535093        1  2018-07-04 17:29:47              3
36535092        1  2018-07-04 17:29:50              3
36535088        1  2018-07-04 17:30:44              3
36535085        1  2018-07-04 17:29:47              3
36535083        1  2018-07-04 17:29:47              3
36535107        1  2018-07-04 17:29:50              3
36535082        1  2018-07-04 17:29:47              3
36535075        1  2018-07-04 17:29:47              3
36535074        1  2018-07-04 17:29:47              3
36520284        1  2018-08-09 16:09:10              1
36406585        1  2018-09-27 12:55:38              1
36406838        1  2018-09-27 12:55:44              1
36523169        1  2018-07-0

In [0]:
train_df = pd.read_csv(path+'train.csv', parse_dates=['auditing_date', 'due_date', 'repay_date'])



In [0]:
train_df['repay_date'] = train_df[['due_date', 'repay_date']].apply(
    lambda x: x['repay_date'] if x['repay_date'] != '\\N' else x['due_date'], axis=1
)

In [0]:
train_df['repay_amt'] = train_df['repay_amt'].apply(lambda x: x if x != '\\N' else 0).astype('float32')
train_df['label'] = (train_df['repay_date'] - train_df['auditing_date']).dt.days


In [0]:
print(train_df[train_df['repay_amt']==0].head())#此时就是刚才定义的，未还款时将应还款时间赋值成实际还款时间了

    user_id  listing_id auditing_date   due_date   due_amt repay_date  \
2    404196     2355665    2018-02-18 2018-03-18  307.9270 2018-03-18   
9    757631     4509276    2018-09-23 2018-10-23  394.3774 2018-10-23   
20   279960     3220453    2018-04-30 2018-05-30  320.5188 2018-05-30   
27   175815     2602604    2018-03-12 2018-04-12  486.1024 2018-04-12   
44   519357     4511127    2018-09-23 2018-10-23  199.7496 2018-10-23   

    repay_amt  label  
2         0.0     28  
9         0.0     30  
20        0.0     30  
27        0.0     31  
44        0.0     30  


In [0]:
train_df.loc[train_df['repay_amt'] == 0, 'label'] = 32

In [0]:
print(train_df[train_df['label']==32].head())

    user_id  listing_id auditing_date   due_date   due_amt repay_date  \
2    404196     2355665    2018-02-18 2018-03-18  307.9270 2018-03-18   
9    757631     4509276    2018-09-23 2018-10-23  394.3774 2018-10-23   
20   279960     3220453    2018-04-30 2018-05-30  320.5188 2018-05-30   
27   175815     2602604    2018-03-12 2018-04-12  486.1024 2018-04-12   
44   519357     4511127    2018-09-23 2018-10-23  199.7496 2018-10-23   

    repay_amt  label  
2         0.0     32  
9         0.0     32  
20        0.0     32  
27        0.0     32  
44        0.0     32  


In [0]:
import gc
clf_labels = train_df['label'].values
amt_labels = train_df['repay_amt'].values
del train_df['label'], train_df['repay_amt'], train_df['repay_date']
gc.collect()
train_due_amt_df = train_df[['due_amt']]
train_num = train_df.shape[0]

In [0]:
test_df = pd.read_csv(path+'test.csv', parse_dates=['auditing_date', 'due_date'])
sub = test_df[['listing_id', 'auditing_date', 'due_amt']]
df = pd.concat([train_df, test_df], axis=0, ignore_index=True)


In [0]:
df.sample(10)

,user_id,listing_id,auditing_date,due_date,due_amt
861181,778706,4521876,2018-09-25,2018-10-25,447.3024
291831,114226,3366801,2018-05-11,2018-06-11,269.9802
673289,185872,2407641,2018-02-24,2018-03-24,340.2717
767063,210427,2772029,2018-03-26,2018-04-26,192.3113
71410,50760,3083894,2018-04-19,2018-05-19,1337.0216
648985,530469,4023219,2018-07-18,2018-08-18,634.1695
409724,510715,1934316,2018-01-07,2018-02-07,299.9140
778986,5119,3477764,2018-05-21,2018-06-21,144.2334
406892,634327,2085971,2018-01-23,2018-02-23,97.3003
877315,273347,3011605,2018-04-13,2018-05-13,385.8409


001_规则判断记录.ipynb          paipaidai_rule.py
1.baseline.py                   拍拍贷常见问题.doc
GBDT.py                         挺无聊的一个代码lb_8907.163500.py
MagicMirror_baseline_33分类.py  简单的EDA及规则.ipynb


In [0]:
from lightgbm.sklearn import LGBMClassifier


In [0]:
clf = LGBMClassifier(
    learning_rate=0.05,
    n_estimators=10000,
    subsample=0.8,
    subsample_freq=1,
    colsample_bytree=0.8,
    random_state=2019,
     device='gpu',
)

In [0]:
trn_x=[[1,2,3],[2,3,4]]
trn_y=[0,1]

In [0]:
clf.fit(
        trn_x, trn_y, 
        eval_set=[(trn_x, trn_y), (trn_x, trn_y)],
        early_stopping_rounds=100, verbose=5
    )

LightGBMError: ignored

In [0]:
####eda
print(train.head())


   user_id  listing_id auditing_date   due_date   due_amt  repay_date  \
0   748147     3163926    2018-04-25 2018-05-25   72.1167  2018-05-25   
1   672952     3698760    2018-06-09 2018-07-09  258.7045  2018-07-08   
2   404196     2355665    2018-02-18 2018-03-18  307.9270          \N   
3   342769     1994522    2018-01-13 2018-02-13  252.9809  2018-02-13   
4   828139     3602352    2018-06-01 2018-07-01  107.6503  2018-06-25   

  repay_amt  
0   72.1167  
1  258.7045  
2        \N  
3  252.9809  
4  107.6503  


In [0]:
print(train.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 7 columns):
user_id          1000000 non-null int64
listing_id       1000000 non-null int64
auditing_date    1000000 non-null datetime64[ns]
due_date         1000000 non-null datetime64[ns]
due_amt          1000000 non-null float64
repay_date       1000000 non-null object
repay_amt        1000000 non-null object
dtypes: datetime64[ns](2), float64(1), int64(2), object(2)
memory usage: 162.4 MB
None


In [0]:
print(train.describe())

              user_id    listing_id         due_amt
count  1000000.000000  1.000000e+06  1000000.000000
mean    446177.855141  3.797037e+06      451.799866
std     257534.686801  9.424460e+05      514.070251
min          1.000000  1.873193e+06       17.453300
25%     225120.750000  3.046201e+06      160.482407
50%     444321.000000  3.832456e+06      286.740112
75%     664508.250000  4.657050e+06      546.026794
max     924400.000000  5.207722e+06    18827.017578


In [0]:
##更改数据类型节省变量


for each in ['user_id','listing_id']:
  train[each]=train[each].astype('int32')##整型变量比字符串变量节省内存
for each in ['due_amt']:
  train[each]=train[each].astype('float32')


In [0]:
train.shape

(1000000, 7)

In [0]:
train[train['repay_date']=='\\N'].count()

user_id          117192
listing_id       117192
auditing_date    117192
due_date         117192
due_amt          117192
repay_date       117192
repay_amt        117192
dtype: int64

In [0]:
print(117192/1000000)##只有11.72%的标的没有还款，大部分标的都按时还款

0.117192


In [0]:
print(train[train['repay_date']=='\\N']['user_id'].value_counts().sort_values())

133893    0
850134    0
850132    0
850131    0
850130    0
850128    0
850127    0
850126    0
850125    0
850123    0
850122    0
850121    0
850120    0
850119    0
850080    0
850077    0
850264    0
850033    0
850046    0
850045    0
850044    0
850043    0
850135    0
850042    0
850136    0
850138    0
850105    0
850104    0
850103    0
850102    0
         ..
241016    3
189589    3
643672    3
672703    3
696998    3
331230    3
284355    3
264216    3
371498    3
262370    3
551256    3
434748    3
434693    3
86824     3
627973    3
129377    3
44324     3
48142     3
741229    3
83544     3
623101    3
804774    3
593585    4
319907    4
260477    4
259733    4
794221    4
434987    4
271000    4
571576    4
Name: user_id, Length: 823732, dtype: int64


In [0]:
print(train[train['user_id']==241016])

       user_id listing_id auditing_date   due_date     due_amt repay_date  \
274761  241016    4422293    2018-09-12 2018-10-12  833.143616         \N   
501054  241016    3989005    2018-07-13 2018-08-13  677.782715         \N   
610754  241016    4580505    2018-10-03 2018-11-03  332.915985         \N   

       repay_amt  
274761        \N  
501054        \N  
610754        \N  


In [0]:
print(train[train['user_id'].isin([571576,804774])].sort_values(by=['user_id','auditing_date']))

##对于某一个标的，一共被多少人借过，有多少人没还(这些人的特征是什么)
##根据user_id统计人的特征，然join得到相应标的上人特征### 人 时间 标的；例如：一共借了多少次（分全量数据，和滑窗数据特征），每次借的金额处于哪个等级，（需要多借款金额离散化）逾期了多少次

       user_id listing_id auditing_date   due_date     due_amt  repay_date  \
218310  571576    2508866    2018-03-04 2018-04-04   97.300301          \N   
40171   571576    3359022    2018-05-11 2018-06-11  133.931107          \N   
726744  571576    5138561    2018-12-17 2019-01-17  102.102203          \N   
472117  571576    5163044    2018-12-21 2019-01-21  101.344101          \N   
222879  804774    4029352    2018-07-18 2018-08-18  426.815186          \N   
175242  804774    4223087    2018-08-17 2018-09-17  223.651199  2018-09-17   
100999  804774    4623660    2018-10-10 2018-11-10  630.828186          \N   
9160    804774    4776702    2018-10-28 2018-11-28  232.765503          \N   

       repay_amt  
218310        \N  
40171         \N  
726744        \N  
472117        \N  
222879        \N  
175242  223.6512  
100999        \N  
9160          \N  


In [0]:
print(train[train['listing_id']==2508866])

       user_id listing_id auditing_date   due_date    due_amt repay_date  \
218310  571576    2508866    2018-03-04 2018-04-04  97.300301         \N   

       repay_amt  
218310        \N  


In [0]:
print(train.sort_values())